Import Libraries
TO DO
1) Add larger test dataset
2) Add Speed Tests
3) Create Real Polygons for Testing Purpose

In [1]:
import numpy as np
from osgeo import gdal
from scipy import stats
from rasterstats import zonal_stats
from shapely.geometry import Point, Polygon
from osgeo import ogr
import math
from math import floor

In [2]:
## Warning You will need 65 gb to create the memory map for the CU imper sum down data type

In [3]:
Grid_Name  = "data\\nlcd_2011_impervious_2011_edition_2014_10_10\\nlcd_2011_impervious_2011_edition_2014_10_10.img"
memory_map_Name = "data\\nlcd_imp.mmp"
memory_map_Name2 = "data\\nlcd_imp_sum_down.mmp"

r = gdal.Open(Grid_Name)
r.GetGeoTransform()
r.RasterXSize
band = r.GetRasterBand(1)
print(band.GetBlockSize())

##DataType = gdal.GetDataTypeName(DataType)
##print(DataType)

[64, 64]


In [4]:
## calculate Values of the raster
block_size = 100;
ysize_mmp = math.ceil(r.RasterYSize / block_size)*block_size + 0; 
xsize_mmp = math.ceil(r.RasterXSize / block_size)*block_size + 0;
rnsze = (ysize_mmp) * (xsize_mmp);

data = np.memmap(memory_map_Name, dtype='uint8', mode='r+', shape=(ysize_mmp * xsize_mmp))
data_sum = np.memmap(memory_map_Name, dtype='float32', mode='r+', shape=(ysize_mmp * xsize_mmp)) 

In [5]:
## rmp function to turn row and column major into a tiled dataset
## This lowers the 
## into blocks
## Yt = RasterYSize, Xt = RasterXSize

def rmp(Yt,Xt,Y,X,blcksze):
    ##Y += 1; X += 1;
    
    DY = (floor((Y)/blcksze)) * blcksze
 
    dy = Y - DY
 
    DX = (floor((X)/blcksze)) * blcksze

    
    dx = X - DX;

    
    p1 = DX * Yt;
    p2 = DY * blcksze;
    p3 = blcksze * dx ;
    p4 = dy   
    pos = p1 + p2 + p3 + p4;
    return(int(pos));


In [6]:
# Function to read the raster as arrays for the chosen block size.
def create_cp_grid(blcksze, raster, mmp, ysize_mmp, xsize_mmp):
    ds = gdal.Open(raster)
    band = ds.GetRasterBand(1)

    blocks = 0
    ##for y in range(0, 20000, blcksze):
    for y in range(1, band.YSize, blcksze):
        ##print(y/band.YSize)
        if y + blcksze < band.YSize:
            rows = blcksze
        else:
            rows = band.YSize - y
        ##for x in range(0, 20000, blcksze):
        for x in range(0, band.XSize, blcksze):
            if x + blcksze < band.XSize:
                cols = blcksze
            else:
                cols = band.XSize - x
            ##array = band.ReadAsArray(x, y, cols, rows)
            i = list([]);
            for ywi in range(0,rows):
                for xwi in range(0,cols):
                      i.append(rmp(ysize_mmp,xsize_mmp,ywi+0+y,xwi+0+x,blcksze))
            mmp[i] = band.ReadAsArray(x, y, cols, rows).flat
            ##print(i[0])
            del i
            blocks += 1
    band = None
    ds = None


In [29]:
#create_cp_grid(100, Grid_Name, data, ysize_mmp, xsize_mmp)

In [23]:
def tst(y,x,blcksze):
    print( "orig is " + str(band.ReadAsArray(x, y, 1, 1)[0]))
    ##data[1]
    print( "new is " + str(data[rmp(ysize_mmp,xsize_mmp,y,x,blcksze)]))



In [24]:
tst(0,0,100)
tst(99,99,100)
tst(100,100,100)
tst(11290,16899,100)
tst(11295,16899,100)
rmp(ysize_mmp,xsize_mmp,1,1,100)
##rmp(ysize_mmp,xsize_mmp,100,100,100)

orig is [127]
new is 127
orig is [127]
new is 127
orig is [127]
new is 127
orig is [59]
new is 59
orig is [60]
new is 60


101

In [20]:
def crte_sum_down(raster, mmp, mmpsm, blcksze):
    for yw in range(1,raster.RasterYSize):
        print(yw/raster.RasterYSize)
        for xw in range(0,raster.RasterXSize):
            i1 = rmp(ysize_mmp,xsize_mmp,yw,xw,blcksze)
            i2 = rmp(ysize_mmp,xsize_mmp,yw-1,xw,blcksze)
            mmpsm[i1] = mmpsm[i2] +  mmp[i1]

In [19]:
crte_sum_down(r, data, data_sum, 100)

9.576342603232974e-06
1.9152685206465947e-05
2.872902780969892e-05
3.8305370412931895e-05
4.7881713016164866e-05
5.745805561939784e-05
6.703439822263081e-05
7.661074082586379e-05
8.618708342909675e-05
9.576342603232973e-05
0.00010533976863556271
0.00011491611123879568
0.00012449245384202866
0.00013406879644526162
0.00014364513904849459
0.00015322148165172758
0.00016279782425496054
0.0001723741668581935
0.0001819505094614265
0.00019152685206465947
0.00020110319466789243
0.00021067953727112542
0.0002202558798743584
0.00022983222247759135
0.00023940856508082432
0.0002489849076840573
0.0002585612502872903
0.00026813759289052324
0.0002777139354937562
0.00028729027809698917
0.0002968666207002222
0.00030644296330345516
0.0003160193059066881
0.0003255956485099211
0.00033517199111315405
0.000344748333716387
0.00035432467631962004
0.000363901018922853
0.00037347736152608597
0.00038305370412931893
0.0003926300467325519
0.00040220638933578486
0.0004117827319390178
0.00042135907454225085
0.00043093

KeyboardInterrupt: 